# [IAPR 2020:][iapr2020] Lab 2 ‒  Object description

**Authors:** Claire Meyer, Nicolas Furrer, Philipp Schuler
**Due date:** 24.04.2020

[iapr2020]: https://github.com/LTS5/iapr-2020

## Extract relevant data
We first need to extract the `lab-02-data.tar.gz` archive.
To this end, we use the [tarfile] module from the Python standard library.

[tarfile]: https://docs.python.org/3.6/library/tarfile.html

In [ ]:
import tarfile
import os

data_base_path = os.path.join(os.pardir, 'data')
data_folder = 'lab-02-data'
# tar_path = os.path.join(data_base_path, data_folder + '.tar.gz')
# with tarfile.open(tar_path, mode='r:gz') as tar:
#     tar.extractall(path=data_base_path)

## Part 1
In the `lab-02-data/part1` folder, you will find 28x28 grey-scale pictures of handwritten "0" and "1".
These digits have been extracted from MNIST dataset (http://yann.lecun.com/exdb/mnist/).

Your goal is to extract, from each of those images, a 2-dimensional feature vector (i.e. 2 features) and to plot them all on a 2D graph.
If you have chosen good features, the vectors of the "0"'s should nicely cluster in one part of the plane and those of the "1"'s in another.

Please try first the Fourier Descriptors.
You can make several attempts: e.g. with and without invariance to rotation, translation, scaling, etc.
You can also for instance rotate the images and assess the invariance in rotation.

**Note:** for the Fourier descriptors, the u_k signal has to be constructed by following the contour point after point.
Some pre-processing (image binarization, possibly some Mathematical Morphology) might be useful.

Then feel free to try other features, the more you try, the better it will be (for you).

### 1.1 Data visualization

In [ ]:
import skimage.io
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import cv2 as cv

# Load images
data_base_path = os.path.join(os.pardir, 'data')
data_folder = 'lab-02-data'
#  Load zeros
zeros_path = os.path.join(data_base_path, data_folder, 'part1', '0')
zeros_names = [nm for nm in os.listdir(zeros_path) if '.png' in nm]  # make sure to only load .png
zeros_names.sort()  # sort file names
ic = skimage.io.imread_collection([os.path.join(zeros_path, nm) for nm in zeros_names])
zeros_im = skimage.io.concatenate_images(ic)
#  Load ones
ones_path = os.path.join(data_base_path, data_folder, 'part1', '1')
ones_names = [nm for nm in os.listdir(ones_path) if '.png' in nm]  # make sure to only load .png
ones_names.sort()  # sort file names
ic = skimage.io.imread_collection(([os.path.join(ones_path, nm) for nm in ones_names]))
ones_im = skimage.io.concatenate_images(ic)

# Plot images
fig, axes = plt.subplots(2, len(zeros_im), figsize=(12, 3))
for ax, im, nm in zip(axes[0], zeros_im, zeros_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)
for ax, im, nm in zip(axes[1], ones_im, ones_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)

### 1.2 Fourier descriptors
Add your implementation and discussion

In [ ]:
import cv2 as cv
import numpy as np
import math


# def border(img):
#     # Converting from RGB to grayscale if necessary
#     if len(img.shape)==3:
#         img	= cv.cvtColor(src=img, code=cv.COLOR_RGB2GRAY)
        
#     # Converting to binary image
#     _, img = cv.threshold(src=img, thresh=0, maxval=1, type=(cv.THRESH_BINARY | cv.THRESH_OTSU))
    
#     [numrows, numcols]=img.shape
    
#     return row, col


def fd(img, N=None, method="cropped"):
    # fd Computes the Fourier Descriptors of a shape. 
    #   [Z, Nin, m, phi, numrows, numcols] = fd(img, N=contour length, method="cropped")

    # Converting from RGB to grayscale if necessary
    if len(img.shape)==3:
        img	= cv.cvtColor(src=img, code=cv.COLOR_RGB2GRAY)
        
    # Converting to binary image
    _, img = cv.threshold(src=img, thresh=0, maxval=1, type=(cv.THRESH_BINARY | cv.THRESH_OTSU))
    [numrows, numcols]=img.shape
    
    # Extracting the contours
    contours,_ = cv.findContours(image=img, mode=cv.RETR_EXTERNAL, method=cv.CHAIN_APPROX_NONE)
    contours = np.asarray(contours).squeeze()
    
    # Complex periodic signal out of the contours
    y = contours[:,0]
    x = contours[:,1]
    z = x + y*1j;
    Nin = z.size;
    
    # Assigning default arg
    if N is None:
        N = Nin;

    # Processing to get the fft
    Z = np.fft.fft(z);

    # Magic to get the correct signal length
    if Nin < N:
        dst = img.copy()
        cv.resize(img, dst, fx=2, fy=2, interpolation=cv.INTER_LINEAR)
        Z, Nin, _, _, _, _ = fd(dst, N, method)
    elif Nin > N:
        i = math.ceil(N/2)

        if method=="cropped":
            Z=np.concatenate((Z[:i],Z[-i:]))
        elif method=="padded":
            Z[i:-i]=0
        else:
            raise ValueError(f"Incorrect 'method' : {method}.")

    m = np.absolute(Z)
    phi = np.angle(Z)
    
    return Z, Nin, m, phi, numrows, numcols
    
    
def ifd(Z, Nin, numrows, numcols, Nout=None):
    # IFD Computes the inverse Fourier Descriptors of a complex signal. 
    #   [img] = ifd(Z, Nin, numrows, numcols, Nout = Z length)

    # Initialization
    N = Z.size

    # Assigning default arg
    if Nout is None:
        Nout = N
    
    # Magic to get the correct signal length
    if N != Nout:        
        if N < Nout:
            i = math.ceil(N/2)
            Z = np.concatenate((Z[:i],np.zeros(Nout-N),Z[i:]))
        elif N > Nout:
            i = math.ceil(Nout/2)
            Z = np.concatenate((Z[:i],Z[-i:]))

    # Processing
    z = np.fft.ifft(Z)
    z = z*Nout/Nin 
    x = z.real
    y = z.imag
    row = y.astype(np.uint8)
    col = x.astype(np.uint8)
    
    img = np.zeros((numrows, numcols), np.uint8)
    
    for xi,yi in zip(col, row):
        if (xi>=0 & yi>=0 & xi<numcols & yi<numrows):
            img[xi,yi] = 255
    
    return img

def afd(img):
    Z, _, _, _, _, _ = fd(img)
    Z = Z/Z[1]
    Z = Z[2:-1]
    m = np.absolute(Z)
    return m


### 1.3 Additional method(s)
Add your implementation and discussion

In [ ]:
# preprocess

# Plot images
fig, axes = plt.subplots(2, len(zeros_im), figsize=(12, 3))
for ax, im, nm in zip(axes[0], zeros_im, zeros_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)
for ax, im, nm in zip(axes[1], ones_im, ones_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)
    


    
def morphologic(list_img):
    kernel = np.zeros((3,3), np.uint8)
    cv.circle(img=kernel, center=(1,1), radius=1, color=255, thickness=-1)
    result = list_img.copy()
    for im, i in zip(list_img, range(list_img.size)):
        im_t = cv.inRange(im, 10,255)
        im_t = im_t/255
        copy_im = im_t.copy()
        result[i] = cv.morphologyEx(copy_im, cv.MORPH_OPEN, kernel, iterations=1)
        
    return result


zeros_m = morphologic(zeros_im)
ones_m = morphologic(ones_im)

# Plot images
fig, axes = plt.subplots(2, len(zeros_m), figsize=(12, 3))
for ax, im, nm in zip(axes[0], zeros_m, zeros_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)
for ax, im, nm in zip(axes[1], ones_m, ones_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)

# Count number of peak per axis with a value minimal of peak_value
def peak_on_axis(list_img, peak_value):
    
    peak_in_x = np.zeros(10)
    peak_in_y = np.zeros(10)
    for im, i in zip(list_img, range(list_img.size)):
        
        im_t = cv.inRange(im, 20,255)
        im_t = im_t/255

        for x in range(im[0,:].size):
            if sum(im_t[x,:]) > peak_value:
                peak_in_x[i] = peak_in_x[i] + 1

        for y in range(im[:,0].size):
            if sum(im_t[:,y]) > peak_value:
                peak_in_y[i] = peak_in_y[i] + 1
                
    return peak_in_x, peak_in_y

# Plot number corresponding of each point
def plot_number(nb_peak_x,nb_peak_y):
    for x,y,i in zip(nb_peak_x, nb_peak_y, range(nb_peak_x.size)):
        plt.text(x,y, str(i), color="black", fontsize=15)

# Computation
peak_value = 5
nb_peak_x_0, nb_peak_y_0 = peak_on_axis(zeros_im,peak_value)
nb_peak_x_1, nb_peak_y_1 = peak_on_axis(ones_im,peak_value)

# Plot
fig, ax = plt.subplots()
plt.title(f"Peak height : {peak_value}")
ax.plot(nb_peak_x_0, nb_peak_y_0, "bo", label='0')
ax.plot(nb_peak_x_1, nb_peak_y_1, "ro", label='1')
ax.legend(loc='upper left')
plot_number(nb_peak_x_0, nb_peak_y_0)
plot_number(nb_peak_x_1, nb_peak_y_1)


#### Minimum area oriented rectangle and closed loops
Two additional methods to extract features from the images are the ratio of a oriented rectangle around the white opints of the images and the number of closed loops. 
The function below extract these features and returns different informations:
 - the two features we want
 - the images of the rectangles for debug purposes
 - the number of contours (can also be used as faeture, but less precise)


The rectangle ratio calculation is done this way:
 - Take the original and find the contours.
 - Then only for the biggest contour, find the minimum area rectangle around the contour
 - Calculate the width/height ratio of this contour. This is the first feature we'll use
 
The closed loops are found with these steps:
 - Threshold the image to obtain a simple binary image
 - With the `connectedComponentsWithStats()` function, extract the number of closed shapes.
 - As the background, and the drawing are a distinct shape, we subtract these from the returned number, thus getting the number of closed loops
 
We then return the needed informations
 

In [ ]:
# returns the ratio of the rotated rectangle. This shows if it is elongated or not, thus if it is close to a 1 or a 0.
def getRatio(im):
       
    contours,_ = cv.findContours(im, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    drawing = np.zeros((im.shape[0], im.shape[1], 3), dtype=np.uint8)
    cv.drawContours(drawing, contours, 0, [255,255,255])
    
    minRect = [None]*len(contours)
    
    max_contour_area = 0
    for i, c in enumerate(contours):
        cv.drawContours(drawing, contours, i, [255,255,255])
        if max_contour_area < cv.contourArea(c): # take index 0 as it is the biggest contour
            max_contour_area = cv.contourArea(c)
            minRect = cv.minAreaRect(c)
            (x, y), (width, height), angle = minRect
            box = cv.boxPoints(minRect)
            box = np.intp(box) #np.intp: Integer used for indexing (same as C ssize_t; normally either int32 or int64)

    total_contours = i+1;
    cv.drawContours(drawing, [box], 0, [255,0,0])
    if width >= height:
        ratio = width/height
    else:
        ratio = height/width
    
    return ratio, drawing, total_contours


# returns the number of closed loops in the image
def getClosedLoops(im):
    
    # number of centroids, this gives the number of closed loops + 2 (background + first color)
    _, thresh = cv.threshold(im, 1, 255, cv.THRESH_BINARY)
    ret, labels, stats, centroids = cv.connectedComponentsWithStats(cv.bitwise_not(thresh),4)
    
    closed_loops = int(centroids.size/2)-2
    
    return closed_loops


In [ ]:
## rotated rectangle + nb of closed loops

fig, axes = plt.subplots(2, len(zeros_im), figsize=(12, 3))

x_0, y_0 = [], [];
x_1, y_1 = [], [];

for ax, im, nm in zip(axes[0], zeros_im, zeros_names):
    
    ratio, drawing, tc =  getRatio(im)
    cl = getClosedLoops(im)
    
    x_0.append(ratio)
    y_0.append(cl)
    
    ax.imshow(drawing, cmap='gray')
    ax.axis('off')
    ax.set_title(round(ratio,2))
    
for ax, im, nm in zip(axes[1], ones_im, ones_names):
    
    ratio, drawing, tc =  getRatio(im)
    cl = getClosedLoops(im)
    
    x_1.append(ratio)
    y_1.append(cl)
    
    ax.imshow(drawing, cmap='gray')
    ax.axis('off')
    ax.set_title("r="+str(round(ratio,2))+",c="+str(childs))
    
    
fig, ax = plt.subplots()
ax.plot(x_1,y_1,'ro')
ax.plot(x_0,y_0,'bo')
ax.legend(['1','0'])

## Part 2
The `lab-02-data/part2` folder contains grey-scale pictures of handwritten "2" and "3".
Extract the same feature (typically 2 Fourier descriptors) as in part 1 also on these images and plot them on the same graph as the features of the "0" and "1".
Is it possible to discriminate all these 4 digits with a 2-dimensional feature vector?

### 2.1 Data visualization

In [ ]:
#  Load zeros
twos_path = os.path.join(data_base_path, data_folder, 'part2', '2')
twos_names = [nm for nm in os.listdir(twos_path) if '.png' in nm]  # make sure to only load .png
twos_names.sort()  # sort file names
ic = skimage.io.imread_collection([os.path.join(twos_path, nm) for nm in twos_names])
twos_im = skimage.io.concatenate_images(ic)
#  Load ones
threes_path = os.path.join(data_base_path, data_folder, 'part2', '3')
threes_names = [nm for nm in os.listdir(threes_path) if '.png' in nm]  # make sure to only load .png
threes_names.sort()  # sort file names
ic = skimage.io.imread_collection(([os.path.join(threes_path, nm) for nm in threes_names]))
threes_im = skimage.io.concatenate_images(ic)

# Plot images
fig, axes = plt.subplots(2, len(twos_im), figsize=(12, 3))
for ax, im, nm in zip(axes[0], twos_im, twos_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)
for ax, im, nm in zip(axes[1], threes_im, threes_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)

### 2.2 Additional method(s) and conclusion
Add your implementations and discussions here

In [ ]:
#Peak on axis test with number 0 to 3
for peak_value in range(3, 15):
    
    # Computation
    nb_peak_x_0, nb_peak_y_0 = compute_max_on_axis(zeros_im, peak_value)
    nb_peak_x_1, nb_peak_y_1 = compute_max_on_axis(ones_im, peak_value)
    nb_peak_x_2, nb_peak_y_2 = compute_max_on_axis(twos_im, peak_value)
    nb_peak_x_3, nb_peak_y_3 = compute_max_on_axis(threes_im, peak_value)

    # Plot
    fig, ax = plt.subplots()
    plt.title(f"Peak height : {peak_value}")
    ax.plot(nb_peak_x_0, nb_peak_y_0, "bo", label='0')
    ax.plot(nb_peak_x_1, nb_peak_y_1, "ro", label='1')
    ax.plot(nb_peak_x_2, nb_peak_y_2, "go", label='2')
    ax.plot(nb_peak_x_3, nb_peak_y_3, "co", label='3')
    ax.legend(loc='upper left')
    plt.show()

#### 2.2.3 Oriented Rectangle + # of contours

In [ ]:
fig, axes = plt.subplots(2, len(zeros_im), figsize=(12, 3))

x_2, y_2 = [], [];
x_3, y_3 = [], [];

for ax, im, nm in zip(axes[0], twos_im, twos_names):
    
    ratio, drawing, tc =  getRatio(im)
    cl = getClosedLoops(im)
    
    x_2.append(ratio)
    y_2.append(cl)
    
    ax.imshow(drawing, cmap='gray')
    ax.axis('off')
    ax.set_title(round(ratio,2))
    
for ax, im, nm in zip(axes[1], threes_im, threes_names):
    
    ratio, drawing, tc =  getRatio(im)
    cl = getClosedLoops(im)
    
    x_3.append(ratio)
    y_3.append(cl)
    
    ax.imshow(drawing, cmap='gray')
    ax.axis('off')
    ax.set_title("r="+str(round(ratio,2))+",c="+str(childs))
    
fig, ax = plt.subplots()
ax.plot(x_2,y_2,'ro')
ax.plot(x_3,y_3,'bo')
ax.legend(['2','3'])